#### Try to convert input sentences into numeric sequences

In [30]:
import tensorflow as tf

Korpus merupakan kumpulan teks yang tersimpan secara elektronik untuk berbagai kebutuhan penyelidikan dan penelitian, salah satunya dalam pemrosesan bahasa alami (Natural Language Processing).

In previous lab, i use the TextVectorization layer to build vocabulary from my corpus. Itu menghasilkan daftar di mana kata-kata yang lebih sering muncul memiliki indeks yang lebih rendah.

In [31]:
#Sample inputs

sentences=[
    'Ayu cantik',
    'Wahyuni bukan Wahyu',
    'I love me',
    'Am I supernova huh?'
]

#inisialisasi layer
vectorize_layer=tf.keras.layers.TextVectorization()

#Build vocabulary
#tiap kata pada sentences disimpan dalam bentuk indeks
vectorize_layer.adapt(sentences)

#Get the vocabulary
vocabulary=vectorize_layer.get_vocabulary()

#Print the token index
for index, word in enumerate(vocabulary):
    print(index, word)

0 
1 [UNK]
2 i
3 wahyuni
4 wahyu
5 supernova
6 me
7 love
8 huh
9 cantik
10 bukan
11 ayu
12 am


In [33]:
#Memeriksa token pada suatu kalimat
#convert sentence to integer sequences

sample_sentence=[
        'saya makan nasi',
        'i love me',
        'i love nasi padang'
]

sample_sentence2='am i supernova huh? '

#convert
sequences1=vectorize_layer(sample_sentence)
sequences2=vectorize_layer(sample_sentence2)

# for index, word in enumerate(sample_sentence):
print(sequences1)
print("\n",sequences2)

tf.Tensor(
[[1 1 1 0]
 [2 7 6 0]
 [2 7 1 1]], shape=(3, 4), dtype=int64)

 tf.Tensor([12  2  5  8], shape=(4,), dtype=int64)


For a given list of string inputs (such as the 4-item `sentences` list above), you will need to apply the layer to each input. There's more than one way to do this. Let's first use the `map()` method and see the results.

Untuk list of string inputs tertentu (seperti daftar `sentences` berisi 4 item di atas), Anda perlu menerapkan layer tersebut ke setiap masukan. Ada lebih dari satu cara untuk melakukannya. Mari kita gunakan metode `map()` terlebih dahulu dan lihat hasilnya.

In [26]:
#convert the list to tf.data.Dataset
#membuat daftar dari kalimat menjadi suatu dataset
#setiap elemen dari dataset adalah satu kalimat dari sentences
sentences_dataset=tf.data.Dataset.from_tensor_slices(sentences)

#mapping function untuk convert each inputs to integer sequence
sequences_sentence=sentences_dataset.map(vectorize_layer)

for sentences, sequences_sentence in zip(sentences, sequences_sentence):
    print(f"{sentences} ----> {sequences_sentence}")



Ayu cantik ----> [11  9]
Wahyuni bukan Wahyu ----> [ 3 10  4]
I love me ----> [2 7 6]
Am I supernova huh? ----> [12  2  5  8]


## Padding


index for padding --> [0]

#### Post-padding

In [35]:
sequences_post=vectorize_layer(sentences)

print('Input: ')
print(sentences)
print()

print('Output: ')
print(sequences_post)

Input: 
['Ayu cantik', 'Wahyuni bukan Wahyu', 'I love me', 'Am I supernova huh?']

Output: 
tf.Tensor(
[[11  9  0  0]
 [ 3 10  4  0]
 [ 2  7  6  0]
 [12  2  5  8]], shape=(4, 4), dtype=int64)
